In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.distributions as dist
import torch
import copy
import matplotlib.pyplot as plt
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import Callback
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from scripts.utils import ScaleData, train_keys
from scripts.AutoEncoder import AutoEncoderDataset
from tqdm import tqdm

import itertools
import seaborn as sns
#%matplotlib notebook
from tqdm import tqdm

In [2]:
# do we have a gpu 
gpu = torch.cuda.is_available()

if gpu:
    device = torch.cuda.current_device()
    print(f'GPU device: {device}')
else: 
    print('No GPU')

GPU device: 0


In [3]:
train_data_path = "/share/rcifdata/jbarr/UKAEAGroupProject/data/train_data_clipped.pkl"
train_data = AutoEncoderDataset(train_data_path, columns = train_keys, train = True)
train_data.data = train_data.data.sample(10_000)
train_data.scale()

train_loader = DataLoader(train_data, shuffle = True, batch_size = 1024)

valid_data_path = "/share/rcifdata/jbarr/UKAEAGroupProject/data/valid_data_clipped.pkl"
valid_data = AutoEncoderDataset(valid_data_path, columns = train_keys, train = True)
valid_data.data = valid_data.data.sample(10_000)
valid_data.scale()

valid_loader = DataLoader(valid_data, shuffle = True, batch_size = 1024)

In [4]:
test = "/share/rcifdata/jbarr/UKAEAGroupProject/data/test_data_clipped.pkl"

df_test = pd.read_pickle(test)
df_test = df_test.sample(10_000)
target = df_test['target']
df_test_good = df_test[df_test.target == 1]
df_test_good = df_test_good[train_keys]

df_test_good,_ = ScaleData(df_test_good)

df_test_bad = df_test[df_test.target == 0]
df_test_bad = df_test_bad[train_keys]
df_test_bad,_ = ScaleData(df_test_bad)

df_test_good.describe()

,ane,ate,autor,machtor,x,zeff,gammae,q,smag,alpha,ani1,ati0,normni1,ti_te0,lognustar
count,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03,6.634000e+03
mean,-5.043197e-17,-3.174696e-17,-1.349873e-16,-5.432294e-16,-1.272138e-16,2.694391e-16,7.082902e-16,7.581113e-18,-9.311130e-17,-3.052109e-17,-2.613434e-17,-2.627450e-17,1.691944e-17,-8.095641e-16,-9.462166e-17
std,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00
min,-6.297388e+00,-5.020761e+00,-1.485119e+01,-1.839907e+00,-1.646459e+00,-1.285040e+00,-1.837735e+01,-1.228424e+00,-2.069795e+00,-1.535920e+00,-1.549730e+01,-6.300319e+00,-1.786372e+00,-3.111402e+00,-2.588042e+00
25%,-3.982774e-01,-5.573673e-01,-2.905987e-01,-5.242054e-01,-9.668236e-01,-7.992450e-01,3.722918e-02,-7.421419e-01,-6.425405e-01,-4.631523e-01,-3.389445e-01,-5.483892e-01,-2.689037e-01,-1.006135e-01,-7.183725e-01
50%,-1.920931e-01,-2.603034e-01,-2.905987e-01,-5.242054e-01,-1.373695e-01,-1.578197e-01,3.722918e-02,-2.888421e-01,-3.918279e-01,-3.263465e-01,-1.827159e-01,-2.521140e-01,-2.104291e-01,-1.006135e-01,-1.362269e-01
75%,1.142332e-01,1.936652e-01,-3.188221e-02,1.067301e-01,9.292663e-01,5.101999e-01,3.722918e-02,4.294962e-01,2.581441e-01,7.170666e-02,5.613432e-02,2.770294e-01,-9.568391e-02,-1.006135e-01,5.715206e-01
max,1.317529e+01,8.528607e+00,2.694902e+01,5.408396e+00,1.636621e+00,1.561337e+01,1.294859e+01,7.053241e+00,6.123863e+00,1.606846e+01,1.464891e+01,1.044813e+01,1.946950e+01,2.036157e+01,6.697918e+00


In [5]:
data_good = torch.from_numpy(df_test_good.values).float()
data_bad = torch.from_numpy(df_test_bad.values).float()
#data_good_batch = next(iter(data_good_loader))
# with torch.no_grad():
#     outputs_good = encoder.forward(data_good).sample()
#     outputs_bad = encoder.forward(data_bad).sample()
# #    outputs_good = encoder.forward(data_good).sample().detach().numpy()
    
# plt.figure()
# plt.scatter(outputs_good[:,0], outputs_good[:,1])
# plt.scatter(outputs_bad[:,0], outputs_bad[:,1])

In [ ]:
AE_output = decoder.forward(encoder.forward(data_good).sample()).sample().detach().numpy()
df_ae_output = pd.DataFrame(AE_output, columns = train_keys)
df_ae_output['AE'] = 'Outputs'

df_test_tmp = df_test_good
df_test_tmp['AE'] = 'Inputs'

In [ ]:
df_compare = pd.concat([df_ae_output, df_test_tmp], ignore_index=True)
df_compare_sample = df_compare.sample(10_000)

In [ ]:
for i in train_keys:
    plt.figure()
    x_min = df_compare_sample[i].quantile(0.1)
    x_max = df_compare_sample[i].quantile(0.9)
    sns.histplot(data = df_compare_sample, x = i, hue = "AE", binrange = (x_min, x_max), bins = 100);
    plt.xlabel(i)

# VAE 2

In [6]:
class LinearVAE(nn.Module):
    def __init__(self):
        super(LinearVAE, self).__init__()
        
 
        # encoder
        self.enc1 = nn.Linear(in_features=15, out_features =150)
        self.enc2 = nn.Linear(in_features=150, out_features=75)
        self.enc3 = nn.Linear(in_features=75, out_features =25)
        
        self.mu = nn.Linear(25,3 )
        self.sigma = nn.Linear(25,3)
 
        # decoder 
        self.dec1 = nn.Linear(in_features = 3, out_features = 25)
        self.dec2 = nn.Linear(in_features = 25, out_features = 75)
        self.dec3 = nn.Linear(in_features = 75, out_features = 150)
        self.dec4 = nn.Linear(150, 15)
        
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample
 
    def forward(self, x):
        # encoding
        x = x.float()
        x = F.relu(self.enc1(x.float()))
        x = F.relu(self.enc2(x.float()))
        x = F.relu(self.enc3(x.float()))
        # get `mu` and `log_var`
        mu = self.mu(x) # the first feature values as mean
        log_var = self.sigma(x) # the other feature values as variance
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
 
        # decoding
        z = F.relu(self.dec1(z.float()))
        z = F.relu(self.dec2(z.float()))
        z = F.relu(self.dec3(z.float()))
        
        reconstruction = self.dec4(z.float())
        return reconstruction.float(), mu.float(), log_var.float()

In [7]:
batch_size = 1024
lr = 1e-3
epochs = 50

model = LinearVAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss(reduction = "sum")

In [8]:
def final_loss(MSE_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    MSE = MSE_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE + KLD

In [9]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        #data, _ = data
        data = data.to(device)
        #data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        MSE_loss = criterion(reconstruction.float(), data.float())
        loss = final_loss(MSE_loss, mu, logvar)
        running_loss += loss.item()
        print(loss.item())
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [10]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(valid_data)/dataloader.batch_size)):
            data = data.to(device)
            #data = data.view(data.size(0), -1)
            reconstruction, mu, logvar = model(data)
            MSE_loss = criterion(reconstruction.float(), data.float())
            loss = final_loss(MSE_loss, mu, logvar)
            running_loss += loss.item()
        
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [11]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = fit(model, train_loader)
    val_epoch_loss = validate(model, valid_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")

Epoch 1 of 50


 33%|███▎      | 3/9 [00:01<00:01,  3.10it/s]

13383.109375
20425.330078125
15250.078125


 56%|█████▌    | 5/9 [00:01<00:00,  4.90it/s]

18296.96484375
13009.90625
14765.318359375


...

8539.0078125
8291.490234375
10765.369140625


10it [00:00, 10.99it/s]                      


4798.15234375


10it [00:00, 11.76it/s]                      

Train Loss: 8.4226
Val Loss: 8.8716


In [ ]:
model.to('cpu')
AE_output2,_,_ = model(data_good)
AE_output2 = AE_output2.detach().numpy()
df_ae_output2 = pd.DataFrame(AE_output2, columns = train_keys)
df_ae_output2['AE'] = 'Outputs'

df_test_tmp = df_test_good
df_test_tmp['AE'] = 'Inputs'

df_compare2 = pd.concat([df_ae_output2, df_test_tmp], ignore_index=True)
df_compare_sample2= df_compare2.sample(10_000)

In [ ]:
for i in train_keys:
    plt.figure()
    x_min = df_compare_sample2[i].quantile(0.1)
    x_max = df_compare_sample2[i].quantile(0.9)
    sns.histplot(data = df_compare_sample2, x = i, hue = "AE", binrange = (x_min, x_max), bins = 100);
    plt.xlabel(i)